In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('ionosphere.csv',header=None)
df=df.drop([1], axis=1)
df=df.drop([0], axis=1)
#print(df.corr())
#df.isnull().sum()

In [11]:
target_col = 34
df.loc[df[target_col] == 'g', 34] = 1
df.loc[df[target_col] == 'b', 34] = 0
df[target_col] = df[target_col].astype(int)
df

,2,3,4,5,6,7,8,9,10,11,...,25,26,27,28,29,30,31,32,33,34
0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1
1,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
2,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
3,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
4,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,0.89391,0.13130,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,1
347,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,0.96510,0.03281,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,1
348,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,0.94124,0.01766,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,1
349,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,0.89724,-0.03315,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,1


In [12]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(df[df.columns.difference([34])],df[34], test_size=0.2, random_state=49)


In [55]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
xTrain=scaler.fit_transform(xTrain)
xTest=scaler.transform(xTest)

In [108]:
import numpy as np

class LDA(object):

    def __init__(self, K):
        self.K = K
        self.explained_variance_ = np.array([])
        self.explained_variance_ratio_ = np.array([])
        self.eigenvectors_ = np.array([])
    
    def compute_scatters(self, X, y):
        """
        param X: numpy array of shape (M,N)
        param y: numpy array of shape (M), shows to which class each row of X belongs
        return S_w, S_b: scatter within and scatter between matrices which are of shape (N,N)
        """
        S_w=np.zeros((X.shape[1],X.shape[1]))
        S_b=np.zeros((X.shape[1],X.shape[1]))
        mu=X.mean()
        for i in y.unique():
            mask=(y==i)
            mu_i=X[mask].mean(axis=0)
            mu=X.mean()
            S_w+=(X[mask]-mu_i).T.dot(X[mask]-mu_i)
            S_b+=mask.sum() * ((mu_i-mu).reshape((mu_i-mu).shape[0],1)).dot(((mu_i-mu).reshape((mu_i-mu).shape[0],1)).T)
        return (S_w, S_b)

    
    def fit(self, X, y):
        """
        param X: numpy array of shape (M,N)
        param y: numpy array of shape (M), shows to which class each row of X belongs
        """
        """TODO fit the model,(compute scatter matrices and compute 
        eigenvalues and eigenvectors of S_w^{-1}S_b) and update values of 
        self.explained_variance_, self.explained_variance_ratio_, self.eigenvectors"""
        prod_matrix=np.linalg.inv(self.compute_scatters(X,y)[0]).dot(self.compute_scatters(X,y)[1])
        eigenvalues, eigenvectors =np.linalg.eig(prod_matrix)
        self.explained_variance_= eigenvalues.real
        self.eigenvectors=eigenvectors
        self.explained_variance_ratio_=self.explained_variance_ /self.explained_variance_.sum() 
 

    def transform(self, X):
        """
        param X: numpy array of shape (M,N)
        return X_proj: numpy array of shape (M,K)
        """
        """TODO use self.explained_variance_ratio_ and self.eigenvectors_
        to project X from dimension N to K"""
        if self.eigenvectors.shape[0] == 0:
            raise TypeError("Please fit before calling transform")
        explained_variance_vectors= [(np.abs( self.explained_variance_ratio_[i]), self.eigenvectors[:,i]) for i in range(len(self.explained_variance_ratio_))]
        explained_variance_vectors=sorted(pairs, key=lambda x: x[0], reverse=True)
        w_matrix=explained_variance_vectors[0][1].reshape(len(self.explained_variance_ratio_),1)
        for i in range(1,self.K): 
            w_matrix =np.hstack((w_matrix,explained_variance_vectors[i][1].reshape(len(self.explained_variance_ratio_),1)))
        X_proj = np.array(X.dot(w_matrix.real))
        return X_proj

In [111]:
lda1=LDA(K=4)
lda1.fit(xTrain,yTrain)
lda1.transform(xTrain).shape, xTrain.shape


((280, 4), (280, 32))

In [112]:
lda1.explained_variance_ratio_
expl_var_rat_lda=lda1.explained_variance_ratio_
var_rat1 = []
for var_mem in expl_var_rat_lda:
    var_rat1.append(var_mem)
    if sum(var_rat1)>=0.90:
        break

k1=len(var_rat1)
k1


1